<h1>Proyecto Tecnologías de Búsqueda en la Web</h1>
<h3>Integrantes</h3>
<ul><li>Sebastián Aranda</li><li>Felipe Santander</li></ul>

<h2>Librerías</h2>

In [36]:
import gensim
from gensim import corpora
from gensim.corpora import Dictionary, bleicorpus
from gensim.models import ldamodel
from gensim.models import Phrases

from collections import defaultdict

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.classify import PositiveNaiveBayesClassifier

import matplotlib.pyplot as plt
import numpy as np

from random import randint
from __future__ import division

#import pyLDAvis
import re
import os
import json
import operator

<h2>Creación del Corpus</h2>

<h6>Rutas a Datasets</h6>

In [37]:
roastme_dataset_path = "Dataset/Bullying"
positive_dataset_path = "Dataset/NoBullying"

<h6>Construcción del Corpus</h6>

In [38]:
#Strange symbols
puncts = ".,:;?!()[]{}~+-\"\'#$%&\/*^|"
digits = "0123456789"

#Create bad words list
bad_words = []
with open('badwords.txt','r') as bad_words_file:
    for word in bad_words_file:
        word = word.split('-')[0].strip()
        word = word.replace('\n','').decode('unicode_escape').encode('ascii','ignore')
        if word != '':
            bad_words.append(word)

#Create the corpus
corpus = list()
for name in os.listdir(positive_dataset_path):
    if name.endswith('.json'):
        with open(positive_dataset_path+'/'+name) as f:
            op_json = json.loads(f.read())
            try:
                for child in op_json[1]['data']['children']:    
                    #Extract the comment
                    comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                    #Delete links
                    comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)
                    
                    for sym in puncts:
                        comment_text = comment_text.replace(sym," ")
                    for num in digits:
                        comment_text = comment_text.replace(num," ")
                    
                    tokens_comment = [word for word in comment_text.lower().split()]        
                    corpus.append(tokens_comment)
                        
                    try:
                        for child in child['data']['replies']['data']['children']:
                            comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                            comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)
                    
                            for sym in puncts:
                                comment_text = comment_text.replace(sym," ")
                            for num in digits:
                                comment_text = comment_text.replace(num," ")
                        
                            tokens_comment = [word for word in comment_text.lower().split()]
                            corpus.append(tokens_comment)
                                
                    except Exception as e:
                        pass
                    
            except Exception as e:
                pass

positive_dataset_size = len(corpus)

for name in os.listdir(roastme_dataset_path):
    if name.endswith('.json'):
        with open(roastme_dataset_path+'/'+name) as f:
            op_json = json.loads(f.read())
            try:
                for child in op_json[1]['data']['children']:    
                    
                    #Constraint for dataset size equality
                    if (len(corpus) >= 2*positive_dataset_size):
                        break
                    
                    comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                    comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)
                    
                    for sym in puncts:
                        comment_text = comment_text.replace(sym," ")
                    for num in digits:
                        comment_text = comment_text.replace(num," ")
                    
                    tokens_comment = [word for word in comment_text.lower().split()]    
                    corpus.append(tokens_comment)
                        
                    try:
                        for child in child['data']['replies']['data']['children']:
                            
                            if (len(corpus) >= 2*positive_dataset_size):
                                break
                            
                            comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                            comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)
                    
                            for sym in puncts:
                                comment_text = comment_text.replace(sym," ")
                            for num in digits:
                                comment_text = comment_text.replace(num," ")
                        
                            tokens_comment = [word for word in comment_text.lower().split()]    
                            corpus.append(tokens_comment)
                                
                    except Exception as e:
                        #print(e)
                        pass
                    
            except Exception as e:
                #print(e)
                pass

print("Tamaño del dataset completo:")
print(len(corpus))
#TODO: guardar corpus de texto con pickle

Tamaño del dataset completo:
97718


In [5]:
print(positive_dataset_size)

48859


<h6>Procesamiento del Corpus</h6>

In [39]:
#Remove lone letters
corpus = [[word for word in doc if len(word)>1] for doc in corpus]

#Remove stopwords
corpus = [[word for word in doc if word not in stopwords.words('english')] for doc in corpus]

# Lemmatize all words in documents.
lemmatizer = WordNetLemmatizer()
corpus = [[lemmatizer.lemmatize(word) for word in doc] for doc in corpus]

#For Sentiment Analysis
comments_pro = [" ".join(doc) for doc in corpus]
    
#Get Trigrams
#bigram = Phrases(corpus)
#trigram = Phrases(bigram[corpus])
#for idx in range(len(corpus)):
#    for token in trigram[corpus[idx]]:
#            if '_' in token:
#                corpus[idx].append(token)

<h6>Transformación del Corpus a espacio de vectores</h6>

In [40]:
#Create dictionary
dictionary = corpora.Dictionary(corpus)
#dictionary.filter_extremes(no_below=len(dictionary)*0.001, no_above=0.75)
dictionary.filter_extremes()

#Convert documents to vectors
corpus = [dictionary.doc2bow(doc) for doc in corpus]

#Save corpus and dictionary
corpora.BleiCorpus.serialize('tmp/cyberbullying_corpus.lda-c', corpus)
dictionary.save('tmp/cyberbullying_dictionary.dict')

In [26]:
#Load the dictionary and corpus
if (os.path.exists('tmp/cyberbullying_dictionary.dict') and os.path.exists('tmp/cyberbullying_corpus.lda-c')):
    print('Creating dictionary...')
    dictionary = corpora.Dictionary.load('tmp/cyberbullying_dictionary.dict')
    print('Creating corpus...')
    print('Corpus Size: '+str(len(corpus)))
    corpus = corpora.BleiCorpus('tmp/cyberbullying_corpus.lda-c')
else:
    print("Create the dictionary and corpus first")

Creating dictionary...
Creating corpus...
Corpus Size: 97718


<h1>Características para Clasificación</h1>
<ul>
<li>Porcentaje de pertenencia al tópico generado de LDA en Roastme Dataset</li>
<li>Densidad de Badwords</li>
<li>Polaridad del post utilizando Sentiment Analysis</li>
</ul>

<h3>LDA Features</h3>

<h4>Modelo</h4>

In [41]:
#Initialize a model
#print('Creating Tfidf model...')
#tfidf = gensim.models.TfidfModel(corpus)
#corpus_tfidf = tfidf[corpus]
#lda_model_tfidf = ldamodel.LdaModel(corpus_tfidf, num_topics=2, id2word=dictionary)

#LDA-2
%time lda_model = ldamodel.LdaModel(corpus, num_topics=2, id2word=dictionary)
lda_model.save('tmp/cyberbullying_ldaModel.lda')
lda_model.print_topics(2,40)

CPU times: user 2min 19s, sys: 616 ms, total: 2min 20s
Wall time: 2min 20s


[(0,
  u'0.015*"like" + 0.012*"look" + 0.011*"roast" + 0.009*"fuck" + 0.008*"post" + 0.006*"one" + 0.006*"dick" + 0.006*"guy" + 0.006*"picture" + 0.006*"know" + 0.006*"please" + 0.005*"would" + 0.005*"sure" + 0.005*"get" + 0.004*"forehead" + 0.004*"use" + 0.004*"let" + 0.004*"never" + 0.004*"nice" + 0.004*"see" + 0.004*"fat" + 0.004*"sex" + 0.004*"subreddit" + 0.004*"u" + 0.003*"got" + 0.003*"gay" + 0.003*"love" + 0.003*"comment" + 0.003*"as" + 0.003*"report" + 0.003*"want" + 0.003*"another" + 0.003*"deleted" + 0.003*"say" + 0.003*"bitch" + 0.003*"rule" + 0.003*"user" + 0.003*"probably" + 0.003*"moderator" + 0.003*"dude"'),
 (1,
  u'0.045*"like" + 0.043*"look" + 0.013*"face" + 0.009*"one" + 0.009*"get" + 0.008*"hair" + 0.007*"guy" + 0.007*"make" + 0.006*"would" + 0.006*"girl" + 0.006*"kid" + 0.006*"think" + 0.005*"even" + 0.005*"time" + 0.005*"school" + 0.005*"tell" + 0.005*"go" + 0.005*"year" + 0.005*"got" + 0.004*"thing" + 0.004*"head" + 0.004*"back" + 0.004*"people" + 0.004*"know" +

In [ ]:
#LDA-3
%time lda_model = ldamodel.LdaModel(corpus, num_topics=3, id2word=dictionary)
lda_model.save('tmp/cyberbullying_ldaModel.lda')
lda_model.print_topics(3,40)

In [ ]:
#LDA-5
%time lda_model = ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary)
lda_model.save('tmp/cyberbullying_ldaModel.lda')
lda_model.print_topics(5,40)

In [ ]:
#LDA-10
%time lda_model = ldamodel.LdaModel(corpus, num_topics=10, id2word=dictionary)
lda_model.save('tmp/cyberbullying_ldaModel.lda')
lda_model.print_topics(10,40)

<h4>Creación de características</h4>

In [42]:
id_mostAgressiveTopic = 0; 

lda_features = []
for docbow in corpus:
    lda_features.append(lda_model[docbow][id_mostAgressiveTopic][1])

<h3>Bad Words Features</h3>

In [43]:
bad_words_ids = [word_id for word_id, word in dictionary.iteritems() if word in bad_words]
bad_words_features = []

for docbow in corpus:
        if (len(docbow) > 0):
            bad_words_features.append(sum([freq for word_id,freq in docbow if word_id in bad_words_ids])/len(docbow))
        else:
            bad_words_features.append(0)

<h3>Sentiment Analysis Features</h3>

In [46]:
positive_dataset_size = int(len(corpus)/2)
positive_comments = [comments_pro[i] for i in range(positive_dataset_size)]
negative_comments = [comments_pro[i] for i in range(positive_dataset_size,len(corpus))]

def features(list_of_comments):
    words = list_of_comments.lower().split()
    return dict(('contains(%s)' % w, True) for w in words)

positive_featuresets = list(map(features, negative_comments)) + list(map(features, bad_words))
#positive_featuresets = list(map(features, bad_words))
unlabeled_featuresets = list(map(features, positive_comments))
classifier = PositiveNaiveBayesClassifier.train(positive_featuresets, unlabeled_featuresets)

sentiment_features = []
for doc in comments_pro:
    sentiment_features.append(classifier.classify(features(doc)))

<h1>Validación Cruzada</h1>

<h3>Etiquetas</h3>
<p>Asignar etiqueta segun corpus</p>

In [50]:
positive_dataset_size = int(len(corpus)/2)
y1 = [0 for label in range(positive_dataset_size)]
y2 = [1 for label in range(positive_dataset_size,len(corpus))]
y = y1 + y2

97718


In [51]:
from sklearn.cross_validation import train_test_split
X = zip(lda_features,bad_words_features, sentiment_features)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .7)

from sklearn.neighbors import KNeighborsClassifier
myClassifier = KNeighborsClassifier()
myClassifier.fit(X_train, y_train)
predictions = myClassifier.predict(X_test)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, predictions))

/home/sebastian/anaconda3/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.747423358625
